In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [18, 8]

# DEV: override installed teslakit
import sys
sys.path.insert(0,'../../')

# teslakit
from teslakit.project_site import Site
from teslakit.statistical import CopulaSimulation
from teslakit.mda import MaxDiss_Simplified_NoThreshold


# --------------------------------------
# Site paths and parameters
data_folder = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
site = Site(data_folder, 'KWAJALEIN')

ST = site.pc.site                          # site database
PR = site.params                           # site parameters

# input files
p_hist_r1_params = ST.TCs.hist_r1_params   # TCs parameters inside radius 1
p_hist_r2_params = ST.TCs.hist_r2_params   # TCs parameters inside radius 2

# output files
p_sim_r2_params = ST.TCs.sim_r2_params     # probabilistic simulated storms 
p_mda_r2_params = ST.TCs.mda_r2_params     # MDA classified simulated storms

# parameters
num_sim_rnd = int(PR.TCS.num_simulate_rnd)
num_sel_mda = int(PR.TCS.num_select_mda)


In [2]:
# --------------------------------------
# Probabilistic simulation Historical TCs

# aux function
def FixPareto(var):
    'Fix data. It needs to start at 0 for Pareto adjustment '
    var = var.astype(float)
    mx = np.amax(var)
    aux = mx + np.absolute(var - mx)
    var_pareto = aux - np.amin(aux) + 0.00001
    
    return var_pareto, np.amin(aux)


# use small radius (4º)
xds_TCs_r2_params = xr.open_dataset(p_hist_r2_params)

pmean = xds_TCs_r2_params.pressure_mean.values[:]
pmin = xds_TCs_r2_params.pressure_min.values[:]
gamma = xds_TCs_r2_params.gamma.values[:]
delta = xds_TCs_r2_params.delta.values[:]
vmean = xds_TCs_r2_params.velocity_mean.values[:]


# fix pressure for pareto adjustment (start at 0)
pmean_p, auxmin_pmean = FixPareto(pmean)
pmin_p, auxmin_pmin = FixPareto(pmin)
# join storm parameters for copula simulation
storm_params = np.column_stack(
    (pmean_p, pmin_p, gamma, delta, vmean)
)

# statistical simulate PCs using copulas 
kernels = ['GPareto', 'GPareto', 'ECDF', 'ECDF', 'ECDF']
storm_params_sim = CopulaSimulation(storm_params, kernels, num_sim_rnd)

# store simulated storms - parameters 
xds_TCs_r2_sim_params = xr.Dataset(
    {
        'pressure_mean':(('storm'), storm_params_sim[:,0] + auxmin_pmean),
        'pressure_min':(('storm'), storm_params_sim[:,1] + auxmin_pmin),
        'gamma':(('storm'), storm_params_sim[:,2]),
        'delta':(('storm'), storm_params_sim[:,3]),
        'velocity_mean':(('storm'), storm_params_sim[:,4]),
    },
    coords = {
        'storm':(('storm'), np.arange(num_sim_rnd))    
    },
)
xds_TCs_r2_sim_params.to_netcdf(p_sim_r2_params)

print(xds_TCs_r2_sim_params)
print('\n\nSimulated TCs parameters stored at:\n{0}'.format(
    p_sim_r2_params))



<xarray.Dataset>
Dimensions:        (storm: 100000)
Coordinates:
  * storm          (storm) int64 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables:
    pressure_mean  (storm) float64 1.011e+03 1.015e+03 ... 1.017e+03 1.053e+03
    pressure_min   (storm) float64 1.011e+03 1.016e+03 ... 1.022e+03 1.063e+03
    gamma          (storm) float64 103.5 137.8 111.8 135.8 ... 0.0 101.2 76.4
    delta          (storm) float64 164.7 108.5 164.3 78.27 ... 100.8 73.57 109.9
    velocity_mean  (storm) float64 35.9 21.03 17.52 6.622 ... 23.1 20.56 14.89


Simulated TCs parameters stored at:
/Users/nico/Projects/TESLA-kit/TeslaKit/data/sites/KWAJALEIN/TCs/TCs_sim_r2_params.nc


In [3]:
# --------------------------------------
# MaxDiss classification

# get simulated parameters  
pmean_s = xds_TCs_r2_sim_params.pressure_mean.values[:]
pmin_s = xds_TCs_r2_sim_params.pressure_min.values[:]
gamma_s = xds_TCs_r2_sim_params.gamma.values[:]
delta_s = xds_TCs_r2_sim_params.delta.values[:]
vmean_s = xds_TCs_r2_sim_params.velocity_mean.values[:]

# subset, scalar and directional indexes
data_mda = np.column_stack((pmean_s, pmin_s, vmean_s, delta_s, gamma_s))
ix_scalar = [0,1,2]
ix_directional = [3,4]

centroids = MaxDiss_Simplified_NoThreshold(
    data_mda, num_sel_mda, ix_scalar, ix_directional 
)


# store MDA storms - parameters 
xds_TCs_r2_MDA_params = xr.Dataset(
    {
        'pressure_mean':(('storm'), centroids[:,0]),
        'pressure_min':(('storm'), centroids[:,1]),
        'gamma':(('storm'), centroids[:,2]),
        'delta':(('storm'), centroids[:,3]),
        'velocity_mean':(('storm'), centroids[:,4]),
    },
    coords = {
        'storm':(('storm'), np.arange(num_sel_mda))    
    },
)
xds_TCs_r2_MDA_params.to_netcdf(p_mda_r2_params)

print(xds_TCs_r2_MDA_params)
print('\n\nMaxDiss TCs parameters stored at:\n{0}'.format(
    p_mda_r2_params))



MaxDiss waves parameters: 100000 --> 1000

   MDA centroids: 1000/1000

<xarray.Dataset>
Dimensions:        (storm: 1000)
Coordinates:
  * storm          (storm) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Data variables:
    pressure_mean  (storm) float64 1.126e+03 1.011e+03 ... 1.087e+03 1.06e+03
    pressure_min   (storm) float64 1.158e+03 1.011e+03 ... 1.118e+03 1.076e+03
    gamma          (storm) float64 11.34 34.98 6.622 35.04 ... 22.55 13.32 25.2
    delta          (storm) float64 168.8 342.1 86.11 50.39 ... 170.9 221.1 44.85
    velocity_mean  (storm) float64 107.6 192.3 12.86 0.0 ... 0.0 126.2 64.86


MaxDiss TCs parameters stored at:
/Users/nico/Projects/TESLA-kit/TeslaKit/data/sites/KWAJALEIN/TCs/TCs_MDA_r2_params.nc
